Code to train the network

In [ ]:
num_actions = 4

In [2]:
import numpy as np
import tensorflow as tf

/Users/adam/anaconda3/envs/energy_py/lib/python3.5/importlib/_bootstrap.py:222: RuntimeWarning: compiletime version 3.6 of module 'tensorflow.python.framework.fast_tensor_util' does not match runtime version 3.5
  return f(*args, **kwds)
/Users/adam/anaconda3/envs/energy_py/lib/python3.5/site-packages/h5py/__init__.py:36: FutureWarning: Conversion of the second argument of issubdtype from `float` to `np.floating` is deprecated. In future, it will be treated as `np.float64 == np.dtype(float).type`.
  from ._conv import register_converters as _register_converters


In [3]:
num_actions = 5

In [4]:
q_vals = tf.placeholder(shape=(None, num_actions), dtype=tf.float32)

selected_action = tf.placeholder(shape=(None), dtype=tf.int64)

q_selected_action = tf.reduce_sum(q_vals * tf.squeeze(tf.one_hot(selected_action, num_actions)), 
                                  reduction_indices=1,
                                  keep_dims=True)

# . TODO compare with and without squeeze

Instructions for updating:
keep_dims is deprecated, use keepdims instead


In [5]:
qv = np.arange(15).reshape(3, -1)
a = np.random.randint(low=0, high=num_actions-1, size=(3,1))

with tf.Session() as sess:
    q_vals = sess.run(q_selected_action,
                      {q_vals: qv,
                       selected_action: a})

In [6]:
qv

array([[ 0,  1,  2,  3,  4],
       [ 5,  6,  7,  8,  9],
       [10, 11, 12, 13, 14]])

In [7]:
a

array([[3],
       [1],
       [0]])

In [8]:
q_vals

array([[ 3.],
       [ 6.],
       [10.]], dtype=float32)

#  Second method for selecting q values for actions

Involves using a two dimensional index

https://github.com/carpedm20/deep-rl-tensorflow does this
```
self.outputs_idx = tf.placeholder('int32', [None, None], 'outputs_idx')                                                                      
self.outputs_with_idx = tf.gather_nd(self.outputs, self.outputs_idx)    


self.actions = tf.placeholder('int64', [None], name='action')                                     

actions_one_hot = tf.one_hot(self.actions, self.env.action_size, 1.0, 0.0, name='action_one_hot')

pred_q = tf.reduce_sum(self.pred_network.outputs * actions_one_hot, reduction_indices=1, name='q_acted')

```

# Masking q values

Used for terminal states

In [9]:
cond = tf.placeholder(shape=(None), dtype=tf.bool)
i = tf.placeholder(shape=(None), dtype=tf.float32)  

zeros = tf.zeros_like(i)
new_tensor = tf.where(condition=cond, x=i, y=zeros)

In [10]:
with tf.Session() as sess:
    o = sess.run(new_tensor,
                {cond: np.array([True, False, True]),
                 i: q_vals})

In [11]:
o

array([[ 3.],
       [ 0.],
       [10.]], dtype=float32)

In [12]:
# . compare this with the baselines approach for speed

# Training and learning

In [13]:
#  need to go through baselines again

if double_q:
    online_action = tf.argmax(online_q_values, axis=1)
    
    target_q_values = target_q_values * tf.squeeze(tf.one_hot(online_action, num_actions))
    
    max_q_next_state = tf.reduce_sum(target_q_values, 
                                     reduction_indices=1,
                                     keep_dims=True)
else:
    max_q_next_state = tf.reduce_max(target_q_values, reduction_indices=1, keep_dims=True)

target = reward + self.gamma * max_q_next_state

# error clipping TODO (Deep-rl-tensorflow)
td_error = q_selected - tf.stop_gradient(target)

error = tf.huber_loss(td_error, name='error')

loss = tf.reduce_mean(tf.multiply(priority_weights, error))

# learning_rate_decay TODO (deep-rl-tenso) !!!!!

optimizer = 

if gradient_norm_clip:
    grads_and_vars = optimizer.compute_gradients(loss, var_list=online_variables)
    
    for idx, (grad, var) in enumerate(grads_and_vars):
        
        if grad is not None:
            grads_and_vars[idx] = (tf.clip_by_norm(grad, gradient_norm_clip), var)
        
        train_op = optimizer.apply_gradients(grads_and_vars)

        else:
    train_op = optimizer.minimize(loss, var_list=online_variables)


SyntaxError: invalid syntax (<ipython-input-13-81e2f1918fc3>, line 25)

In [ ]:
# target net copy ops - happy with the method I use already
# key is to decide whether to make the function completely separate from the agent

# ie to just accept two networks, or one, or none etc

tf.group(*copy_ops, scope='copy_op')